# <center>Class 17: Probabilities & Classification </center>

In [ ]:
import os
import datetime
import pandas as pd
import numpy as np
import sys
import patsy
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
import sklearn.metrics as metrics
from sklearn.metrics import brier_score_loss, roc_curve, auc, confusion_matrix, roc_auc_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

### Define helper functions

In [ ]:
def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))
    
def create_coef_matrix(X, model):
    coef_matrix = pd.concat(
        [pd.DataFrame(X.columns),pd.DataFrame(model.coef_.flatten())], axis = 1
    )
    coef_matrix.columns = ['variable', 'coefficient']
    coef_matrix.iloc[-1] = ['Intercept', model.intercept_.flatten()[0]]
    return coef_matrix

def cv_summary(lambdas, C_values, model):
    d = {'lambdas': lambdas, 
         'C_values': C_values, 
         'mean_cv_score': model.scores_[1].mean(axis = 0)}
    return(pd.DataFrame(data=d))

"""def create_roc_plot(y_true, y_pred):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    all_coords = pd.DataFrame({
        'fpr': fpr,
        'tpr': tpr,
        'thresholds': thresholds
    })
    
    plot = ggplot(all_coords, aes(x = 'fpr', y = 'tpr')) \
        + geom_line(color=color[0], size = 0.7) \
        + geom_area(position = 'identity', fill = 'mediumaquamarine', alpha = 0.3) \
        + xlab("False Positive Rate (1-Specifity)") \
        + ylab("True Positive Rate (Sensitivity)") \
        + geom_abline(intercept = 0, slope = 1,  linetype = "dotted", color = "black") \
        + scale_y_continuous(limits = (0, 1), breaks = seq(0, 1, .1), expand = (0, 0.01)) \
        + scale_x_continuous(limits = (0, 1), breaks = seq(0, 1, .1), expand = (0.01, 0)) \
        + theme_bw()
    return(plot)
"""

def create_roc_plot(y_true, y_pred):
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(6, 6))
    
    # Plot ROC curve line
    ax.plot(fpr, tpr, color='k', linewidth=0.7)
    
    # Fill area under curve
    ax.fill_between(fpr, tpr, alpha=0.3, color='white')
    
    # Add diagonal dotted line
    ax.plot([0, 1], [0, 1], linestyle=':', color='black')
    
    # Set axis labels
    ax.set_xlabel('False Positive Rate (1-Specificity)')
    ax.set_ylabel('True Positive Rate (Sensitivity)')
    
    # Set axis limits and ticks
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_xticks(np.arange(0, 1.1, 0.1))
    ax.set_yticks(np.arange(0, 1.1, 0.1))
    
    # Style similar to theme_bw()
    ax.grid(True, linestyle='-', alpha=0.2)
    ax.set_facecolor('white')
    for spine in ax.spines.values():
        spine.set_color('black')
    
    # Adjust layout
    plt.tight_layout()
    
    return fig, ax


def sigmoid_array(x):
    return(1 / (1 + np.exp(-x)))

def generate_fold_prediction(model, X, fold, param_index):
    fold_coef = model.coefs_paths_[1][fold,param_index,:]
    return(sigmoid_array(np.dot(X, np.transpose(fold_coef)[:-1]) +  np.transpose(fold_coef)[-1]))

"""def create_loss_plot(all_coords, optimal_threshold, curr_exp_loss):
    all_coords_copy = all_coords.copy()
    all_coords_copy['loss'] = (all_coords_copy.false_pos*FP + all_coords_copy.false_neg*FN)/all_coords_copy.n
    
    t = optimal_threshold
    l = curr_exp_loss
    
    plot = ggplot(all_coords_copy, aes(x = 'thresholds', y = 'loss')) + \
        geom_line(color=color[0], size=0.7) + \
        scale_x_continuous(breaks = seq(0, 1.1, by = 0.1)) + \
        coord_cartesian(xlim=(0,1))+ \
        geom_vline(xintercept = t , color = color[0] ) + \
        annotate(geom = "text", x = t - 0.01, y= max(all_coords_copy.loss) - 0.4,
                 label="best threshold: " + str(round(t,2)),
                 colour=color[1], angle=90, size = 7) +\
        annotate(geom = "text", x = t + 0.06, y= l,\
                 label= str(round(l, 2)), size = 7) +\
        theme_bw()
    return(plot)"""

def create_loss_plot(all_coords, optimal_threshold, curr_exp_loss):
    # Create copy and calculate loss
    all_coords_copy = all_coords.copy()
    all_coords_copy['loss'] = (all_coords_copy.false_pos*FP + all_coords_copy.false_neg*FN)/all_coords_copy.n
    
    t = optimal_threshold
    l = curr_exp_loss

    # Create figure and axis
    fig, ax = plt.subplots(figsize=(6, 5))

    # Plot loss line
    ax.plot(all_coords_copy['thresholds'], all_coords_copy['loss'], 
            color= 'k', linewidth=0.7)

    # Add vertical line at optimal threshold
    ax.axvline(x=t, color = 'k')

    # Add annotations
    ax.text(t - 0.04, max(all_coords_copy.loss) - 0.5,
            f"best threshold: {t:.2f}", 
            color = 'k', 
            rotation=90, 
            fontsize = 9)
    
    ax.text(t + 0.06, l,
            f"{l:.2f}",
            fontsize = 9)

    # Set x-axis ticks and limits
    ax.set_xticks(np.arange(0, 1.1, 0.1))
    ax.set_xlim(0, 1)

    # Style similar to theme_bw()
    ax.grid(True, linestyle='-', alpha=0.2)
    ax.set_facecolor('white')
    ax.set_xlabel('threshold')
    ax.set_ylabel('loss')
    for spine in ax.spines.values():
        spine.set_color('black')

    # Adjust layout
    plt.tight_layout()

    return fig, ax



"""def create_roc_plot_with_optimal(all_coords, optimal_threshold):
    all_coords_copy = all_coords.copy()
    all_coords_copy['sp'] = all_coords_copy.true_neg/all_coords_copy.neg
    all_coords_copy['se'] = all_coords_copy.true_pos/all_coords_copy.pos
    
    best_coords = all_coords_copy[all_coords_copy.thresholds == optimal_threshold]
    sp = best_coords.sp.values[0]
    se = best_coords.se.values[0]

    plot = ggplot(all_coords_copy, aes(x = 'sp', y = 'se')) +\
        geom_line(color=color[0], size=0.7) +\
        scale_y_continuous(breaks = seq(0, 1.1, by = 0.1)) +\
        scale_x_reverse(breaks = seq(0, 1.1, by = 0.1)) +\
        geom_point(data = pd.DataFrame({'sp': [sp], 'se': [se]})) +\
        annotate(geom = "text", x = sp, y = se + 0.03,
                 label = str(round(sp, 2)) + ', ' + str(round(se, 2)), size = 7) +\
        theme_bw()
    return(plot)
"""
def create_roc_plot_with_optimal(all_coords, optimal_threshold):
    # Create copy and calculate metrics
    all_coords_copy = all_coords.copy()
    all_coords_copy['sp'] = all_coords_copy.true_neg/all_coords_copy.neg
    all_coords_copy['se'] = all_coords_copy.true_pos/all_coords_copy.pos
    
    # Get optimal point
    best_coords = all_coords_copy[all_coords_copy.thresholds == optimal_threshold]
    sp = best_coords.sp.values[0]
    se = best_coords.se.values[0]
    
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(6, 6))
    
    # Plot ROC curve
    ax.plot(all_coords_copy['sp'], all_coords_copy['se'],
            color='k', linewidth=0.9)
    
    # Add optimal point
    ax.scatter([sp], [se], color='k', s = 100)
    
    # Add text annotation
    ax.text(sp, se + 0.03,
            f"{sp:.2f}, {se:.2f}",
            fontsize = 9,
            ha='center')
    ax.text(sp - 0.02, se - 0.18,
            'specificity (TNR) \n& sensitivity (TPR) \nat the best threshold',
            fontsize = 9,
            ha='center'
           )
    
    # Set axis ticks and limits
    ax.set_yticks(np.arange(0, 1.1, 0.1))
    ax.set_xticks(np.arange(0, 1.1, 0.1))
    ax.set_xlabel('specificity')
    ax.set_ylabel('sensitivity')
    
    # Reverse x-axis
    ax.set_xlim(1, 0)
    
    # Style similar to theme_bw()
    ax.grid(True, linestyle='-', alpha=0.2)
    ax.set_facecolor('white')
    for spine in ax.spines.values():
        spine.set_color('black')
    
    # Adjust layout
    plt.tight_layout()
    plt.show()
    return fig, ax

## Data

In [ ]:
path = os.path.join(os.pardir, 'data', 'bisnode_firms_clean.csv') # this will produce a path with the right syntax for your operating system
path

In [ ]:
# DATA IMPORT - FROM FILE
df = pd.read_csv(path)

In [ ]:
df.head()

In [ ]:
df.columns.tolist()

### Define variable sets 

In [ ]:
rawvars = ["curr_assets", "curr_liab", "extra_exp", "extra_inc", "extra_profit_loss", "fixed_assets",
              "inc_bef_tax", "intang_assets", "inventories", "liq_assets", "material_exp", "personnel_exp",
              "profit_loss_year", "sales", "share_eq", "subscribed_cap"]

qualityvars = ["balsheet_flag", "balsheet_length", "balsheet_notfullyear"]

engvar = ["total_assets_bs", "fixed_assets_bs", "liq_assets_bs", "curr_assets_bs",
            "share_eq_bs", "subscribed_cap_bs", "intang_assets_bs", "extra_exp_pl",
            "extra_inc_pl", "extra_profit_loss_pl", "inc_bef_tax_pl", "inventories_pl",
            "material_exp_pl", "profit_loss_year_pl", "personnel_exp_pl"]

engvar2 = ["extra_profit_loss_pl_quad", "inc_bef_tax_pl_quad",
             "profit_loss_year_pl_quad", "share_eq_bs_quad"]

engvar3 = []
for col in df.columns:
    if col.endswith('flag_low') or col.endswith('flag_high') or col.endswith('flag_error') or col.endswith('flag_zero'):
        engvar3.append(col)

d1 =  ["d1_sales_mil_log_mod", "d1_sales_mil_log_mod_sq",
         "flag_low_d1_sales_mil_log", "flag_high_d1_sales_mil_log"]

hr = ["female", "ceo_age", "flag_high_ceo_age", "flag_low_ceo_age",
        "flag_miss_ceo_age", "ceo_count", "labor_avg_mod",
        "flag_miss_labor_avg", "foreign_management"]

In [ ]:
df.ind2_cat.value_counts().sort_index()

See `patsy` formula language [here](https://patsy.readthedocs.io/en/latest/formulas.html), treatment of categorical variables [here](https://patsy.readthedocs.io/en/latest/categorical-coding.html). 

In [ ]:
#Creat dummy columns from category variables and drop first level
ind2_catmat = patsy.dmatrix("0 + C(ind2_cat)", df, return_type="dataframe")



1. "0 +" at the start means "no intercept" - it suppresses the automatic inclusion of an intercept term (which would be a column of 1s)

2. "C(ind2_cat)" means:
   1. C() is the categorical term operator
   2. it will convert the 'ind2_cat' column into dummy variables
   3. it creates one column for each unique category in ind2_cat

In [ ]:
ind2_catmat

Question: why are we dropping the first level? 

In [ ]:
ind2_catmat = ind2_catmat.drop(['C(ind2_cat)[26]'], axis=1)

In [ ]:
df.m_region_loc.value_counts()

In [ ]:
m_region_locmat = patsy.dmatrix("0 + C(m_region_loc)",df, return_type="dataframe")
m_region_locmat = m_region_locmat.drop(['C(m_region_loc)[Central]'], axis=1)

In [ ]:
df.urban_m.value_counts()

In [ ]:
urban_mmat = patsy.dmatrix("0 + C(urban_m)",df, return_type="dataframe")
urban_mmat = urban_mmat.drop(['C(urban_m)[1]'], axis=1)

In [ ]:
# Define X1
basevars = df[["sales_mil_log", "sales_mil_log_sq", "d1_sales_mil_log_mod", "profit_loss_year_pl"]]
X1 = pd.concat([basevars, ind2_catmat], axis=1)

# Define X2
X2additional_vars = df[["fixed_assets_bs", "share_eq_bs","curr_liab_bs", "curr_liab_bs_flag_high", \
                          "curr_liab_bs_flag_error",  "age", "foreign_management"]]
X2 = pd.concat([X1, X2additional_vars], axis=1)

# Define X3
firm = pd.concat([df[["age", "age2", "new"]], ind2_catmat, m_region_locmat, urban_mmat], axis=1)
X3 = pd.concat([df[["sales_mil_log", "sales_mil_log_sq"] + engvar + d1], firm], axis=1)

# Define X4
X4 = pd.concat([df[["sales_mil_log", "sales_mil_log_sq"] + engvar + d1 \
                                 + engvar2 + engvar3 + hr + qualityvars], firm], axis=1)

In [ ]:
# Define X5

#Creat matrix for interactions1 variables
int1mat = patsy.dmatrix("0 + C(ind2_cat):age + C(ind2_cat):age2 + C(ind2_cat):d1_sales_mil_log_mod \
                + C(ind2_cat):sales_mil_log + C(ind2_cat):ceo_age + C(ind2_cat):foreign_management \
                + C(ind2_cat):female + C(ind2_cat):C(urban_m) + C(ind2_cat):labor_avg_mod", 
                        df, return_type="dataframe")

#Drop first level to get k-1 dummies out of k categorical levels 
for col in int1mat.columns:
    if col.startswith('C(ind2_cat)[26]') or col.endswith('C(urban_m)[1]'):
        int1mat = int1mat.drop([col], axis=1)
        
#Creat matrix for interactions2 variables        
int2mat = patsy.dmatrix("0 + sales_mil_log:age + sales_mil_log:female + sales_mil_log:profit_loss_year_pl \
                + sales_mil_log:foreign_management", 
                        df, return_type="dataframe")

X5 = pd.concat([X4, int1mat, int2mat], axis=1)

In [ ]:
# Define logitvars for LASSO
logitvars = pd.concat([X4, int1mat, int2mat], axis=1)

In [ ]:
# Define rfvars for RF (no interactions, no modified features)
rfvars  = pd.concat([df[["sales_mil", "d1_sales_mil_log"] + rawvars + hr + qualityvars], firm], axis=1)

In [ ]:
y = df['default']

In [ ]:
y.mean()

### Simple linear and logistic models

**Simplest model: X1**

In [ ]:
ols_modelx1 = LinearRegression().fit(X1, y)

In [ ]:
regression_results(y, ols_modelx1.predict(X1))

In [ ]:
ols_modelx1.coef_

In [ ]:
ols_modelx1.coef_.flatten()

In [ ]:
create_coef_matrix(X1, ols_modelx1)

In [ ]:
glm_modelx1 = LogisticRegression(
    solver = "newton-cg", 
    max_iter = 1000, 
    penalty = None, 
    random_state = 20250224).fit(X1, y)
regression_results(y, glm_modelx1.predict(X1))

- Negative R² means that your model is performing worse than a horizontal line (the mean of the target variable). In other words, your model's predictions are worse than simply guessing the mean value every time.
- Negative explained variance similarly indicates that your model's predictions have more variance than the actual data points themselves.

On the negative variance: see also Wikipedia on [$R^2$](https://en.wikipedia.org/wiki/Coefficient_of_determination).

In [ ]:
glm_modelx1.coef_

In [ ]:
glm_modelx1.coef_.flatten()

In [ ]:
create_coef_matrix(X1, glm_modelx1)

**Model X2**

In [ ]:
glm_modelx2 = LogisticRegression(
    solver="newton-cg", 
    max_iter=1000, 
    penalty=None).fit(X2, y)
regression_results(y, glm_modelx2.predict(X2))

In [ ]:
create_coef_matrix(X2, glm_modelx2)

In [ ]:
mx2 = sm.Logit(y,sm.add_constant(X2)).fit().get_margeff()
print(mx2.summary())

**Baseline model is X4 (all vars, but no interactions)**

In [ ]:
ols_model = LinearRegression().fit(X4, y)
regression_results(y, ols_model.predict(X4))

In [ ]:
create_coef_matrix(X4, ols_model)

In [ ]:
glm_model = LogisticRegression(
    solver = "newton-cg",
    max_iter = 1000, 
    penalty= None, verbose = 3).fit(X4, y)
regression_results(y, glm_model.predict(X4))

In [ ]:
create_coef_matrix(X4, glm_model)

In [ ]:
# Get marginal effects
m = sm.Logit(y,sm.add_constant(X4)).fit().get_margeff()

In [ ]:
type(m)

In [ ]:
print(m.summary())

<br> 

**Keep significant variables only**

In [ ]:
t = m.summary().tables[1]

In [ ]:
type(t)

In [ ]:
t

In [ ]:
columns = pd.DataFrame(t.data).iloc[0].tolist()

In [ ]:
df_marginal_effects = pd.DataFrame(t.data, columns = columns).iloc[1:].astype(
    {'dy/dx': float, 'P>|z|': float})
df_marginal_effects[df_marginal_effects['P>|z|'] <= 0.05].sort_values(by = 'dy/dx', ascending = False).reset_index(drop = True)

### Split data into train and holdout

In [ ]:
index_train, index_holdout= train_test_split(
    df.index.values, train_size=round(0.8*len(df.index)), random_state=42)

y_train = y.iloc[index_train]
y_holdout = y.iloc[index_holdout]

In [ ]:
print('Total')
print(df['default'].value_counts(normalize=True), '\n')
print('Train')
print(df.iloc[index_train]['default'].value_counts(normalize=True), '\n')
print('Holdout')
print(df.iloc[index_holdout]['default'].value_counts(normalize=True), '\n')

### Predictions using cross-validations

####  <font color = ' blue'> A digression into optimization algorithms </font>


    
Our task is to find the best parameters that give us the least error in predicting the output. We call the function that calculates this error a **cost function**  or a **loss function**, and our goal is to minimize the error in order to get the best-predicted output. This cost function is usally **quadratic**, either globally with one optimum (simple case) or locally with multiple local optima and one global optima somewhere along the sets of parameters.
- *newton-cg*: It uses a quadratic function minimalization and finds global optimum swiftly. Drawbacks: computationally expensive and stops at the first saddle point.
- *newton-cholesky*: It uses a newton-cg algorithm with a Cholesky decomposition of the Hessian matrix. This solver is more efficient than "newton-cg" for large datasets, as it requires less memory and computational resources. However, it may not converge for some datasets.
- *liblinear*: It is a linear classification that supports logistic regression and linear support vector machine. It applies automatic parameter selection (a.k.a L1 Regularization) and it’s recommended when you have high dimension dataset (recommended for solving large-scale classification problems).
- *lbfgs*: It is a simplified newton-cg which performs better on a limited dataset. It, however, may not converge to anything. 
‘newton-cholesky’, 
- *sag* (stochastic average descent): It optimizes the sum of a finite number of smooth convex functions. Like stochastic gradient (SG) methods, the SAG method's iteration cost is independent of the number of terms in the sum. However, by incorporating a memory of previous gradient values, the SAG method achieves a faster convergence rate than black-box SG methods. Drawbacks: it only supports L2 regularization and it is less practical for large dataset for its memory consumption. 
- *saga* (stochastic average descent): A sag-alternative which supports L2 regularization.

More [here](https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-definitions)

Supported penalties by solver:

- *lbfgs*: L2
- *liblinear*: L1, L2
- *newton-cg*: L2
- *newton-cholesky*: L2
- *sag*: L2
- *saga*: elasticnet, L1, L2

#### specify cross-validation method

In [ ]:
k = KFold(n_splits = 5, shuffle = True, random_state = 20250224)

#### Logistic regression

In [ ]:
# no regularisation needed so setting the paremeter to very high value
Cs_value_logit = [1e20]

We are using `LogisticRegressionCV`, a logistic regression with built-in cross-validation.

Note: Each of the values in Cs describes the inverse of regularization strength. If Cs is as an int, then a grid of Cs values are chosen in a logarithmic scale between 1e-4 and 1e4. Like in support vector machines, *smaller values specify stronger regularization*.

In [ ]:
logit_model_vars = [X1.iloc[index_train], X2.iloc[index_train], X3.iloc[index_train], X4.iloc[index_train], X5.iloc[index_train]]

logit_models = dict()
CV_RMSE_folds = dict()

In [ ]:
for i in range(len(logit_model_vars)):
    print(datetime.datetime.now(), f'Running regression {i}...')
    LRCV_brier = LogisticRegressionCV(
        Cs = Cs_value_logit, 
        cv = k, # simply the number of folds
        refit = True, 
        scoring = 'neg_brier_score', 
        solver = "newton-cg", 
        tol=1e-7, 
        random_state = 20250224)
    logit_models['X'+str(i+1)] = LRCV_brier.fit(logit_model_vars[i], y_train)
    
    # Calculate RMSE on test for each fold
    CV_RMSE_folds['X'+str(i+1)] = np.sqrt(-1*(logit_models['X'+str(i+1)].scores_[1])).flatten()

In [ ]:
np.sqrt(-1*logit_models['X5'].scores_[1].flatten())

In [ ]:
pd.DataFrame(CV_RMSE_folds)

#### Lasso logit

In [ ]:
normalized_logitvars = pd.DataFrame(StandardScaler().fit_transform(logitvars.iloc[index_train]))
normalized_logitvars.columns = logitvars.columns

In [ ]:
lambdas=list(10**np.arange(-1,-4.01, -1/3))
n_obs = normalized_logitvars.shape[0]*4/5
Cs_values = [1/(l*n_obs) for l in lambdas]

In [ ]:
Cs_values # the strength of the regularization -> supressing unimportant variables

Training for ***accuracy***

In [ ]:
logLasso = LogisticRegressionCV(
    Cs = Cs_values, 
    penalty = 'l1', # L1 makes it lasso
    cv = k, 
    refit = True, 
    scoring = 'accuracy', 
    solver = 'liblinear',
    random_state = 20250224)

In [ ]:
%%time
logit_models["LASSO"] = logLasso.fit(normalized_logitvars, y_train)

In [ ]:
cv_summary_lasso = cv_summary(lambdas, Cs_values, logit_models["LASSO"])
cv_summary_lasso

In [ ]:
best_lambda = cv_summary_lasso.sort_values('mean_cv_score', ascending = False).iloc[0,0]
best_lambda

In [ ]:
create_coef_matrix(normalized_logitvars, logit_models["LASSO"])

Training for ***Brier-score*** (aka RMSE)

In [ ]:
#refit with negative brier score so we have RMSE values for the same cv split

logLasso_brier = LogisticRegressionCV(
    Cs = Cs_values, 
    penalty = 'l1', 
    cv = k, 
    refit = True, 
    scoring = 'neg_brier_score', 
    solver = "liblinear", 
    random_state = 20250224)
logLasso_brier_fitted = logLasso_brier.fit(normalized_logitvars, y_train)

In [ ]:
for i, l in enumerate(lambdas):
    if l == best_lambda:
        best_lambda_i = i
        CV_RMSE_folds['LASSO'] = np.sqrt(-1*(logLasso_brier_fitted.scores_[1][:,i])).tolist()

In [ ]:
pd.DataFrame(CV_RMSE_folds)

### AUC using no loss fn

**Calculate AUC for each folds**

In [ ]:
CV_AUC_folds = dict()

**refit logit models with AUC so we have AUC values for the same cv split**

Training for ***AUC***

In [ ]:
%%time
for i in range(len(logit_model_vars)):
    print(datetime.datetime.now(), f'Running regression {i}...')
    LRCV_auc = LogisticRegressionCV(
        Cs = Cs_value_logit, 
        cv = k, 
        refit = True, 
        scoring = 'roc_auc', 
        solver= "newton-cg", 
        tol = 1e-7, 
        random_state = 20250224)
    LRCV_auc_fit = LRCV_auc.fit(logit_model_vars[i], y_train)
    # Calculate AUC on test for each fold
    CV_AUC_folds['X'+str(i+1)] = LRCV_auc_fit.scores_[1][:,0].tolist()

In [ ]:
pd.DataFrame(CV_AUC_folds)

In [ ]:
%%time
#refit with AUC so we have AUC values for the same cv split

logLasso_auc = LogisticRegressionCV(
    Cs=Cs_values, 
    penalty='l1', # this makes it lasso
    cv=k, 
    refit=True, scoring='roc_auc', 
    solver="liblinear", 
    random_state = 20250224)

logLasso_auc_fitted = logLasso_auc.fit(normalized_logitvars, y_train)

CV_AUC_folds['LASSO'] = logLasso_auc_fitted.scores_[1][:,best_lambda_i].tolist()

In [ ]:
pd.DataFrame(CV_AUC_folds)

**For each model: average RMSE and average AUC for models**

In [ ]:
CV_RMSE = dict()
CV_AUC = dict()
nvars = dict()

In [ ]:
for key in logit_models:
    CV_RMSE[key] = np.mean(CV_RMSE_folds[key])
    CV_AUC[key] = np.mean(CV_AUC_folds[key])  

In [ ]:
CV_AUC

In [ ]:
for key in logit_models:
    if key != 'LASSO':
        nvars[key] = logit_models[key].n_features_in_
    else:
        nvars[key] = sum(x != 0 for x in logit_models[key].coef_[0].tolist())

In [ ]:
for k, v in nvars.items():
    print(f'{k.ljust(5)}: {str(v).rjust(3)}')

<br>

We have 6 models, (5 logit and the logit lasso). For each we have a 5-CV RMSE and AUC.
We pick our preferred model based on that.

In [ ]:
logit_summary1 = np.transpose(pd.DataFrame.from_dict([nvars,CV_RMSE,CV_AUC], orient='columns'))
logit_summary1.columns = ['Number of predictors', 'CV RMSE', 'CV AUC']

In [ ]:
logit_summary1

Take best model and estimate RMSE on holdout
X4, X5 and LASSO are practically the same - go with the simplest model.

The `predict_proba` method returns *probababilites*, as opposed to the `predict` method (see below), which returns *classes* (trues and falses). 

In [ ]:
best_model = logit_models['X4']
best_model_X_holdout = X4.iloc[index_holdout]
logit_predicted_probabilities_holdout = best_model.predict_proba(best_model_X_holdout)[:,1]
best_rmse_holdout = np.sqrt(metrics.mean_squared_error(y_holdout, logit_predicted_probabilities_holdout))
round(best_rmse_holdout, 3)

In [ ]:
logit_predicted_probabilities_holdout

In [ ]:
fpr, tpr, thresholds = roc_curve(y_holdout, logit_predicted_probabilities_holdout)

In [ ]:
df_ = pd.DataFrame({
    'fpr': fpr,
    'tpr': tpr,
    'thresholds': thresholds
})

In [ ]:
df_.iloc[:10]

In [ ]:
df_.iloc[-10:]

In [ ]:
plt.figure(figsize = (6,6))
ax = sns.lineplot(
    data = df_,
    x = fpr, 
    y = tpr)
ax.set_title('ROC curve for the best model (X4)')
ax.set_xlabel("False positive rate (1 - Specificity)"), 
ax.set_ylabel("True positive rate (Sensitivity)")
ax.set_xticks([x/10 for x in range(0,11,1)])
ax.set_yticks([x/10 for x in range(0,11,1)])
ax.plot(df_.fpr, df_.fpr, color = 'k', linewidth = 1)
ax.grid(True, linestyle=':');

**Confusion table with different tresholds**

**Default**: 0.5 is used as threshold to convert probabilities to binary classes. The `predict` method uses 0.5 as threshold to split positives and negatives.

In [ ]:
logit_class_prediction = best_model.predict(best_model_X_holdout)

In [ ]:
logit_class_prediction

In [ ]:
values, counts = np.unique(logit_class_prediction.tolist(), return_counts=True)

In [ ]:
values

In [ ]:
counts

In [ ]:
print(values[0],' (no default): ',counts[0])
print(values[1],' (default): ',counts[1])

confusion matrix: summarize different type of errors and successfully predicted cases   
positive = "yes": explicitly specify the positive case

In [ ]:
cm_object1 = confusion_matrix(y_holdout, logit_class_prediction, labels=[0,1])
cm1 = pd.DataFrame(cm_object1, 
    index=['Actual no default', 'Actual default'], 
    columns=['Predicted no default', 'Predicted default'])
cm1

Applying **different thresholds**

- 0.5 same as before

In [ ]:
holdout_prediction = np.where(logit_predicted_probabilities_holdout < 0.5, 0, 1)
cm_object1b = confusion_matrix(y_holdout, holdout_prediction, labels=[0,1])
cm1b = pd.DataFrame(cm_object1b, 
    index=['Actual no default', 'Actual default'], 
    columns=['Predicted no default', 'Predicted default'])
cm1b

In [ ]:
print(f'TPR: {round(cm1b.iloc[1,1] / cm1b.iloc[1].sum(), 3)}')
print(f'FPR: {round(cm1b.iloc[0,1] / cm1b.iloc[0].sum(), 3)}')

In [ ]:
216/(216+574)

- a sensible choice: **mean of predicted probabilities**

In [ ]:
mean_predicted_default_prob = np.mean(logit_predicted_probabilities_holdout)
round(mean_predicted_default_prob, 3)

In [ ]:
holdout_prediction = np.where(logit_predicted_probabilities_holdout < mean_predicted_default_prob, 0, 1)
cm_object2 = confusion_matrix(y_holdout, holdout_prediction, labels=[0,1])
cm2 = pd.DataFrame(cm_object2, 
    index=['Actul no defaul', 'Actual default'], 
    columns=['Predicted no default', 'Predicted default'])
cm2

In [ ]:
print(f'TPR: {round(cm2.iloc[1,1] / cm2.iloc[1].sum(), 3)}')
print(f'FPR: {round(cm2.iloc[0,1] / cm2.iloc[0].sum(), 3)}')

### AUC using a loss function

**Loss function**: relative cost of of a false negative classification (as compared with a false positive classification)

In [ ]:
FP = 1
FN = 10
cost = FN/FP

# the prevalence, or the proportion of cases in the population (n.cases/(n.controls+n.cases))
prevelance = y_train.sum()/len(y_train)
print(f'Prevelance: {prevelance:,.3f}')
print(f'Simple best threshold estimation: {FP/(FP+FN):,.3f}.')

**Draw ROC Curve and find optimal threshold with loss function**

Instread of with the simple threshold calculation we are running a search to find the thresholds where the loss is the smallest for each model for every fold. 

The optimal cut-off is the threshold that *maximizes the distance to the identity (diagonal) line*.

In [ ]:
for i, v in enumerate(logit_models):
    print(i, v)

In [ ]:
k = KFold(n_splits = 5, shuffle = True, random_state = 20250224)

In [ ]:
best_thresholds_cv = dict()
expected_loss_cv = dict()
fold5_threshold = dict()
fold5_expected_loss = dict()
fold5_all_coords = dict()

for i, model_name in enumerate(logit_models):
    best_thresholds = []
    expected_loss =[]
    
    if model_name != 'LASSO':
        X = logit_model_vars[i]
        c_index = 0
    else:
        X = normalized_logitvars
        c_index = best_lambda_i
    
    fold = 0
    
    for train_index, test_index in k.split(X):
        X_fold = X.iloc[test_index,:]
        y_fold = y_train.iloc[test_index]
        
        pred_fold = generate_fold_prediction(logit_models[model_name], X_fold, fold, c_index)
    
        false_pos_rate, true_pos_rate, thresholds = roc_curve(y_fold, pred_fold)

        # this is where we pick the best threshold for minimal loss at each model for every single fold
        optimal_threshold = sorted(
            list(
                zip(
                    np.abs(
                        true_pos_rate + (1 - prevelance)/(cost * prevelance)*(1-false_pos_rate)
                    ),
                    thresholds
                )
            )
            , key=lambda i: i[0], reverse=True)[0][1]
        
        best_thresholds.append(optimal_threshold)
        threshold_prediction = np.where(pred_fold < optimal_threshold, 0, 1)
        
        tn, fp, fn, tp = confusion_matrix(y_fold, threshold_prediction, labels=[0,1]).ravel()

        # expected loss for each model for every single fold
        curr_exp_loss = (fp*FP + fn*FN)/len(y_fold)
        expected_loss.append(curr_exp_loss)
        fold = fold+1

    # best threshold and expected loss for each model (averaging out the folds)
    best_thresholds_cv[model_name] = np.mean(best_thresholds)
    expected_loss_cv[model_name] = np.mean(expected_loss)

    # best threshold and expected loss for each model for fold5 ONLY
    fold5_threshold[model_name] = optimal_threshold
    fold5_expected_loss[model_name] = curr_exp_loss

    all_coords = pd.DataFrame({
        'false_pos': false_pos_rate*sum(y_fold == 0),
        'true_pos': true_pos_rate*sum(y_fold == 1),
        'false_neg': sum(y_fold == 1) - true_pos_rate*sum(y_fold == 1),
        'true_neg': sum(y_fold == 0) - false_pos_rate*sum(y_fold == 0),
        'pos': sum(y_fold == 1),
        'neg': sum(y_fold == 0),
        'n': len(y_fold),
        'thresholds': thresholds
    })
    
    fold5_all_coords[model_name] = all_coords

In [ ]:
fold5_all_coords['X1']

In [ ]:
logit_summary2 = pd.DataFrame(best_thresholds_cv.items(),columns=['Model', 'Avg of optimal thresholds'])
logit_summary2['Threshold for Fold5'] = fold5_threshold.values()
logit_summary2['Avg expected loss'] = expected_loss_cv.values()
logit_summary2['Expected loss for Fold5'] = fold5_expected_loss.values()

In [ ]:
logit_summary2

### Loss plot based on Fold5 in CV

In [ ]:
model_to_plot = 'X1'

In [ ]:
fold5_all_coords[model_to_plot]

In [ ]:
fold5_threshold[model_to_plot]

In [ ]:
fold5_expected_loss[model_to_plot]

In [ ]:
create_loss_plot(
    fold5_all_coords[model_to_plot], 
    fold5_threshold[model_to_plot], 
    fold5_expected_loss[model_to_plot]);

#### ROC plot plot based on Fold5 in CV

In [ ]:
model_to_plot = 'X1'
create_roc_plot_with_optimal(fold5_all_coords[model_to_plot], fold5_threshold[model_to_plot]);

#### Pick best model based on average expected loss

### X4

In [ ]:
best_logit_optimal_treshold = best_thresholds_cv["X4"]
best_logit_optimal_treshold

**Get expected loss on holdout**

In [ ]:
holdout_treshold = np.where(logit_predicted_probabilities_holdout < best_logit_optimal_treshold, 0, 1)
tn, fp, fn, tp = confusion_matrix(y_holdout, holdout_treshold, labels=[0,1]).ravel()
expected_loss_holdout = (fp*FP + fn*FN)/len(y_holdout)
round(expected_loss_holdout, 3)

In [ ]:
cm_object3 = confusion_matrix(y_holdout, holdout_treshold, labels=[0,1])
cm3 = pd.DataFrame(cm_object3, 
    index=['Actul no defaul', 'Actual default'], 
    columns=['Predicted no default', 'Predicted default'])
cm3

In [ ]:
print(f'TPR: {round(cm3.iloc[1,1] / cm3.iloc[1].sum(), 3)}')
print(f'FPR: {round(cm3.iloc[0,1] / cm3.iloc[0].sum(), 3)}')

### Prediction with random forest 

In [ ]:
rfvars_train = rfvars.iloc[index_train]
rfvars_holdout = rfvars.iloc[index_holdout]

In [ ]:
rfvars_train.columns

### Probability forest

A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. The sub-sample size is controlled with the `max_samples` parameter if `bootstrap = True` (default), otherwise the whole dataset is used to build each tree.
- `max_features`: number of features to consider when looking for the best split
- `min_samples_fit`: the minimum number of samples required`to split an internal node

In [ ]:
grid = {'max_features': [5, 6, 7],
        'criterion':['gini'],
        'min_samples_split': [11, 16]}

In [ ]:
prob_forest = RandomForestClassifier(
    random_state=20250224, 
    n_estimators=50, # this should be higher, but it is kept low in order to control runtime on class
    oob_score=True)

prob_forest_grid = GridSearchCV(
    prob_forest, 
    grid, 
    cv = 5, 
    refit='accuracy', # Refit an estimator using the best found parameters on the whole dataset.
                      # For multiple metric evaluation, this needs to be a str denoting the scorer 
                      # that would be used to find the best parameters for refitting the estimator at the end.
    scoring = ['accuracy', 'roc_auc', 'neg_brier_score'], 
    n_jobs=-1)

In [ ]:
%%time
prob_forest_fit = prob_forest_grid.fit(rfvars_train, y_train)

**CV summary table**

In [ ]:
prob_forest_fit.cv_results_

Our grid has 3x2=6 elements. 

In [ ]:
cv_accuracy = np.zeros([6])
for i in range(5):
    cv_accuracy = cv_accuracy + prob_forest_fit.cv_results_['split' + str(i) + '_test_accuracy']
cv_accuracy = cv_accuracy/5
cv_accuracy

In [ ]:
np.max(cv_accuracy)

In [ ]:
prob_forest_fit.best_score_

In [ ]:
cv_auc = np.zeros([6])
for i in range(5):
    cv_auc = cv_auc + prob_forest_fit.cv_results_['split' + str(i) + '_test_roc_auc']
cv_auc = cv_auc/5
cv_auc

In [ ]:
cv_rmse = np.zeros([6])
for i in range(5):
    cv_rmse = cv_rmse +np.sqrt(-1*(prob_forest_fit.cv_results_['split' + str(i) + '_test_neg_brier_score'])).tolist()
cv_rmse = cv_rmse/5
cv_rmse

In [ ]:
prob_forest_cv_results = pd.DataFrame({
    'max_features': prob_forest_fit.cv_results_['param_max_features'],
    'min_samples_split': prob_forest_fit.cv_results_['param_min_samples_split'],
    'cv_accuracy': cv_accuracy,
    'cv_auc': cv_auc,
    'cv_rmse': cv_rmse
})

In [ ]:
prob_forest_cv_results.style.format(
    {
        'cv_accuracy': '{:.5f}',
        'cv_auc': '{:.5f}',
        'cv_rmse': '{:.5f}'
    }
)

**Optimal parameter values**

In [ ]:
best_max_features = prob_forest_fit.best_params_['max_features']
best_min_sample_split = prob_forest_fit.best_params_['min_samples_split']
prob_forest_fit.best_params_

**Average RMSE and AUC over folds**

In [ ]:
prob_forest_best_results = prob_forest_cv_results[
    (prob_forest_cv_results.max_features == best_max_features) & 
    (prob_forest_cv_results.min_samples_split == best_min_sample_split)]
prob_forest_best_results_index = prob_forest_best_results.index.values[0]

CV_RMSE['rf_p'] = prob_forest_best_results.cv_rmse.values[0]
CV_AUC['rf_p'] = prob_forest_best_results.cv_auc.values[0]

In [ ]:
CV_RMSE

In [ ]:
CV_AUC

**Get fold level RMSE and AUC**

In [ ]:
CV_RMSE_folds_rf_p = list()

for i in range(5):
    rmse = np.sqrt(-1*(prob_forest_fit.cv_results_['split' + str(i) + '_test_neg_brier_score'])).tolist()[prob_forest_best_results_index]
    CV_RMSE_folds_rf_p.append(rmse)

CV_RMSE_folds['rf_p'] = CV_RMSE_folds_rf_p

CV_AUC_folds_rf_p = list()

for i in range(5):
    rmse = prob_forest_fit.cv_results_['split' + str(i) + '_test_roc_auc'][prob_forest_best_results_index]
    CV_AUC_folds_rf_p.append(rmse)

CV_AUC_folds['rf_p'] = CV_AUC_folds_rf_p

In [ ]:
pd.DataFrame(CV_AUC_folds)

**Now use loss function and search for best thresholds and expected loss for each folds**

In [ ]:
best_thresholds = list()
expected_loss = list()

fold = 0
for train_index, test_index in k.split(rfvars_train):
    print(f'{datetime.datetime.now()}: fold {fold}...') 
    X_fold = rfvars_train.iloc[test_index,:]
    y_fold = y_train.iloc[test_index]
    
    X_fold_train = rfvars_train.iloc[train_index,:]
    y_fold_train = y_train.iloc[train_index]
    
    prob_forest_best = RandomForestClassifier(
        random_state=20250224, 
        n_estimators=50, # make it higher in practice (default being 100)
        oob_score=True,
        criterion = 'gini', 
        max_features = best_max_features, min_samples_split = best_min_sample_split
    )
    
    prob_forest_best_fold = prob_forest_best.fit(X_fold_train, y_fold_train)
    pred_fold = prob_forest_best_fold.predict_proba(X_fold)[:,1]

    false_pos_rate, true_pos_rate, threshold = roc_curve(y_fold, pred_fold)
    
    best_threshold = sorted(
        list(
            zip(
                np.abs(
                    true_pos_rate + (1 - prevelance)/(cost * prevelance)*(1-false_pos_rate)
                ),
                threshold
            )
        ), 
        key=lambda x: x[0], reverse=True)[0][1]
    
    best_thresholds.append(best_threshold)
    
    threshold_prediction = np.where(pred_fold < best_threshold, 0, 1)
    tn, fp, fn, tp = confusion_matrix(y_fold, threshold_prediction, labels=[0,1]).ravel()
    curr_exp_loss = (fp*FP + fn*FN)/len(y_fold)
    expected_loss.append(curr_exp_loss)
    fold += 1

In [ ]:
for f, t in enumerate(best_thresholds):
    print(f'Fold: {f}, best threshold: {t:,.4f}.')

In [ ]:
for f, l in enumerate(expected_loss):
    print(f'Fold: {f}, smalles loss: {l:,.4f}.')

In [ ]:
fold5_threshold_rf = best_threshold
fold5_expected_loss_rf = curr_exp_loss

all_coords_rf = pd.DataFrame({
    'false_pos': false_pos_rate*sum(y_fold == 0),
    'true_pos': true_pos_rate*sum(y_fold == 1),
    'false_neg': sum(y_fold == 1) - true_pos_rate*sum(y_fold == 1),
    'true_neg': sum(y_fold == 0) - false_pos_rate*sum(y_fold == 0),
    'pos': sum(y_fold == 1),
    'neg': sum(y_fold == 0),
    'n': len(y_fold),
    'thresholds': threshold
})

all_coords_rf

In [ ]:
expected_loss_cv['rf_p'] = np.mean(expected_loss)
best_thresholds_cv['rf_p'] = np.mean(best_thresholds)

In [ ]:
rf_summary = pd.DataFrame(
    {'CV RMSE': [round(CV_RMSE['rf_p'], 3)],
     'CV AUC': [round(CV_AUC['rf_p'], 3)],
     'Avg of optimal thresholds': [round(best_thresholds_cv['rf_p'], 3)],
     'Threshold for Fold5': [round(best_threshold, 3)],
     'Avg expected loss': [round(expected_loss_cv['rf_p'], 3)],
     'Expected loss for Fold5': [round(curr_exp_loss, 3)]})

rf_summary

**Plots based on Fold5 in CV**

In [ ]:
create_loss_plot(all_coords_rf, fold5_threshold_rf, fold5_expected_loss_rf);

In [ ]:
create_roc_plot_with_optimal(all_coords_rf, fold5_threshold_rf)

**Take model to holdout and estimate RMSE, AUC and expected loss**

In [ ]:
prob_forest_fit_best = prob_forest_fit.best_estimator_
rf_predicted_probabilities_holdout = prob_forest_fit_best.predict_proba(rfvars_holdout)[:,1]
rmse_rf = np.sqrt(mean_squared_error(y_holdout, rf_predicted_probabilities_holdout))
round(rmse_rf, 3)

**ROC AUC  on holdout**

In [ ]:
auc_rf = roc_auc_score(y_holdout, rf_predicted_probabilities_holdout)
round(auc_rf, 3)

**Expected loss on holdout**

In [ ]:
holdout_treshold = np.where(rf_predicted_probabilities_holdout < best_thresholds_cv['rf_p'], 0, 1)
tn, fp, fn, tp = confusion_matrix(y_holdout, holdout_treshold, labels=[0,1]).ravel()
expected_loss_holdout = (fp*FP + fn*FN)/len(y_holdout)
round(expected_loss_holdout, 3)

In [ ]:
import datetime
print(datetime.datetime.now())

### Getting probalbilities and classifications from RandomforestClassifier

- classes

In [ ]:
prob_forest_best_fold.predict(rfvars_holdout)

- probabilities

In [ ]:
prob_forest_best_fold.predict_proba(rfvars_holdout)

## Final results

In [ ]:
nvars['rf_p'] = len(rfvars.columns)

In [ ]:
summary_results = pd.DataFrame({"Model": list(nvars.keys()),
                              "Number of predictors": list(nvars.values()),
                              "CV RMSE": list(CV_RMSE.values()),
                              "CV AUC": list(CV_AUC.values()),
                              "CV threshold": list(best_thresholds_cv.values()),
                              "CV expected Loss": list(expected_loss_cv.values())
                               })

In [ ]:
summary_results.style.format(
    {
        'CV threshold': '{:.4f}',
        'CV AUC': '{:.4f}',
        'CV RMSE': '{:.4f}',
        'CV expected Loss': '{:.4f}'
    }
)